In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values 
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        backcast, forecast = theta[:, :self.backcast_size], theta[:, -self.forecast_size:]
        return backcast, forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        backcast = x
        forecast = torch.zeros(x.shape[0], x.shape[1]).to(x.device)
        for block in self.blocks:
            b, f = block(backcast)
            backcast = backcast - b
            forecast = forecast + f
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
    print(f'Test Loss: {test_loss / len(test_loader):.4f}')

# Make predictions
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale
    print(f"Predicted future values: {future_forecast[:5]}")  # Display a few predictions


c:\Users\Shadow\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 119])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Shadow\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([28, 1])) that is different to the input size (torch.Size([28, 119])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [0/100], Loss: 0.5018
Epoch [10/100], Loss: 0.0812
Epoch [20/100], Loss: 0.0287
Epoch [30/100], Loss: 0.0163
Epoch [40/100], Loss: 0.0062
Epoch [50/100], Loss: 0.0094
Epoch [60/100], Loss: 0.0180
Epoch [70/100], Loss: 0.0581
Epoch [80/100], Loss: 0.0051
Epoch [90/100], Loss: 0.0009
Test Loss: 0.1650
Predicted future values: [[ 76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.66014  76.66014  76.66014  76.66014  76.66014  76.66014
   76.66014  76.6601

c:\Users\Shadow\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7, 119])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Step 1: Identify and clean non-numeric values
# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Step 2: Drop rows with NaN values
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Block and Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)  # Only output theta_size (forecasting steps)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        return theta  # Return only forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        forecast = torch.zeros(x.shape[0], 1).to(x.device)  # Initialize forecast shape [batch_size, 1]
        for block in self.blocks:
            f = block(x)  # No backcast subtraction, only forecast aggregation
            forecast = forecast + f  # Aggregate forecasts
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting (single future value)
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
    print(f'Test Loss: {test_loss / len(test_loader):.4f}')



# Make predictions
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale

    # Convert to DataFrame
    forecast_df = pd.DataFrame(future_forecast, columns=["Forecast"])

    # Save to CSV
    forecast_df.to_csv('future_forecast.csv', index=False)

    print(f"Predictions saved to 'future_forecast.csv'")



Epoch [0/100], Loss: 0.4177
Epoch [10/100], Loss: 0.0482
Epoch [20/100], Loss: 0.0321
Epoch [30/100], Loss: 0.0065
Epoch [40/100], Loss: 0.0069
Epoch [50/100], Loss: 0.0157
Epoch [60/100], Loss: 0.0021
Epoch [70/100], Loss: 0.0026
Epoch [80/100], Loss: 0.0019
Epoch [90/100], Loss: 0.0169
Test Loss: 0.1409
Predictions saved to 'future_forecast.csv'


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Step 1: Identify and clean non-numeric values
# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Step 2: Drop rows with NaN values
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Block and Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)  # Only output theta_size (forecasting steps)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        return theta  # Return only forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        forecast = torch.zeros(x.shape[0], 1).to(x.device)  # Initialize forecast shape [batch_size, 1]
        for block in self.blocks:
            f = block(x)  # No backcast subtraction, only forecast aggregation
            forecast = forecast + f  # Aggregate forecasts
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting (single future value)
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    all_preds = []
    all_targets = []
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
        all_preds.extend(y_pred.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

    print(f'Test Loss (MSE): {test_loss / len(test_loader):.4f}')

# Calculate Evaluation Metrics
all_preds = np.array(all_preds).flatten()
all_targets = np.array(all_targets).flatten()

mae = mean_absolute_error(all_targets, all_preds)
mse = mean_squared_error(all_targets, all_preds)
rmse = np.sqrt(mse)
r2 = r2_score(all_targets, all_preds)

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R² (R-squared): {r2:.4f}')

# Make predictions and save to CSV
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale

    # Convert to DataFrame
    forecast_df = pd.DataFrame(future_forecast, columns=["Forecast"])

    # Save to CSV
    forecast_df.to_csv('future_forecast.csv', index=False)

    print(f"Predictions saved to 'future_forecast.csv'")


Epoch [0/100], Loss: 0.4299
Epoch [10/100], Loss: 0.0551
Epoch [20/100], Loss: 0.0225
Epoch [30/100], Loss: 0.0125
Epoch [40/100], Loss: 0.0048
Epoch [50/100], Loss: 0.0024
Epoch [60/100], Loss: 0.0042
Epoch [70/100], Loss: 0.0214
Epoch [80/100], Loss: 0.0405
Epoch [90/100], Loss: 0.0017
Test Loss (MSE): 0.1489
Mean Absolute Error (MAE): 0.2738
Mean Squared Error (MSE): 0.1552
Root Mean Squared Error (RMSE): 0.3939
R² (R-squared): 0.8344
Predictions saved to 'future_forecast.csv'


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the sliding window data
file_path = r'C:\Users\Shadow\Desktop\GIT_X\GIT_PE\thesis\Sliding Window\modi_data.csv'
data = pd.read_csv(file_path)

# Step 1: Identify and clean non-numeric values
# Replace problematic non-numeric values with NaN and convert to float
data = data.apply(pd.to_numeric, errors='coerce')

# Step 2: Drop rows with NaN values
data = data.dropna()

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(data.iloc[:, :-1])  # Features
y = scaler.fit_transform(data.iloc[:, -1].values.reshape(-1, 1))  # Target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the N-Beats Block and Model
class NBeatsBlock(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size):
        super(NBeatsBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, theta_size)  # Only output theta_size (forecasting steps)
        )
        self.backcast_size = input_size
        self.forecast_size = theta_size

    def forward(self, x):
        theta = self.fc(x)
        return theta  # Return only forecast

class NBeatsModel(nn.Module):
    def __init__(self, input_size, hidden_size, theta_size, num_blocks):
        super(NBeatsModel, self).__init__()
        self.blocks = nn.ModuleList([NBeatsBlock(input_size, hidden_size, theta_size) for _ in range(num_blocks)])
    
    def forward(self, x):
        forecast = torch.zeros(x.shape[0], 1).to(x.device)  # Initialize forecast shape [batch_size, 1]
        for block in self.blocks:
            f = block(x)  # No backcast subtraction, only forecast aggregation
            forecast = forecast + f  # Aggregate forecasts
        return forecast

# Model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 256  # Hidden layer size
theta_size = 1  # Output size for forecasting (single future value)
num_blocks = 3  # Number of blocks in the model

# Initialize the model
model = NBeatsModel(input_size=input_size, hidden_size=hidden_size, theta_size=theta_size, num_blocks=num_blocks)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    all_preds = []  
    all_targets = []  
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()
        all_preds.extend(y_pred.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

    print(f'Test Loss (MSE): {test_loss / len(test_loader):.4f}')

# Calculate Evaluation Metrics
all_preds = np.array(all_preds).flatten()
all_targets = np.array(all_targets).flatten()

mae = mean_absolute_error(all_targets, all_preds)
mse = mean_squared_error(all_targets, all_preds)
rmse = np.sqrt(mse)
r2 = r2_score(all_targets, all_preds)

print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R² (R-squared): {r2:.4f}')

# Make predictions and save to CSV
with torch.no_grad():
    future_forecast = model(X_test_tensor)
    future_forecast = scaler.inverse_transform(future_forecast.numpy())  # Inverse scale

    # Convert to DataFrame
    forecast_df = pd.DataFrame(future_forecast, columns=["Forecast"])

    # Save to CSV
    forecast_df.to_csv('future_forecast.csv', index=False)

    print(f"Predictions saved to 'future_forecast.csv'")


Epoch [0/100], Loss: 0.4408
Epoch [10/100], Loss: 0.0417
Epoch [20/100], Loss: 0.0277
Epoch [30/100], Loss: 0.0068
Epoch [40/100], Loss: 0.0062
Epoch [50/100], Loss: 0.0050
Epoch [60/100], Loss: 0.0010
Epoch [70/100], Loss: 0.0055
Epoch [80/100], Loss: 0.0105
Epoch [90/100], Loss: 0.0196
Test Loss (MSE): 0.1592
Mean Absolute Error (MAE): 0.2827
Mean Squared Error (MSE): 0.1631
Root Mean Squared Error (RMSE): 0.4038
R² (R-squared): 0.8260
Predictions saved to 'future_forecast.csv'


# Explanation

### Sliding Window Effect:
We'r using a **sliding window** approach to create input sequences from the dataset. For each sliding window, the model uses past values to predict the next future value. This approach naturally reduces the number of predictions we can make because the initial time steps are used to "build" the input windows, and we cannot predict for them.

### Train-Test Split:
We split the data into a **training set** and a **test set**. Used the `train_test_split` function with `test_size=0.2`, which means that **20% of the data** was used as the test set for prediction.

### Let's calculate step-by-step:

1. **Original Dataset**: Started with 1336 rows.

2. **Sliding Window Effect**:
   - Suppose the window size (how many past time steps used to predict the next one) is \( N \). You lose \( N-1 \) rows in total when generating sliding windows.
   - For example, if \( N = 4 \) (a typical window size), then 3 rows are lost.

   After applying the sliding window, the number of rows becomes:
   \[
   1336 - (N-1) = 1336 - 3 = 1333
   \]
   So, now 1333 rows after applying the sliding window.

3. **Train-Test Split**:
   - Used `test_size=0.2`, meaning 20% of the data is used for testing (and prediction) while 80% is used for training. Let's calculate the test size:
   \[
   \text{Test Set Size} = 1333 \times 0.2 = 266.6 \approx 266 \text{ rows}
   \]
   So, the test set contains 266 rows.

4. **Loss of Initial Windows**: The first \( N-1 \) rows of the test set cannot be used for prediction because we need a sliding window to create input sequences.
   - With \( N = 4 \), we lose 3 rows from the test set, which gives:
   \[
   266 - (N-1) = 266 - 3 = 263 \text{ rows}
   \]
   
This explains why the output `future_forecast.csv` contains **263 rows** of predictions.

### Summary of the Calculation:
- Original dataset: 1336 rows.
- After sliding window (with \( N = 4 \)): 1333 rows.
- After 20% test split: 266 rows in the test set.
- After accounting for the sliding window on the test set: 263 rows available for prediction.

This is why the final output CSV contains 263 rows of predictions. 
